In [9]:
import robotic as ry
import numpy as np

from time import sleep

import math
import sys



def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)

def komo_onestate_hold(C, robot, m_object):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addControlObjective([], 0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)


def komo_onestate_move(C, robot, m_object, goal):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder= 1, enableCollisions=True)
    komo.addModeSwitch([0,1], ry.SY.stable, [robot, m_object], True) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    komo.addObjective([1, 2] , ry.FS.positionDiff, [m_object, goal], ry.OT.eq) 
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)

    

def rrt_solver(k, start_state, end_state, trial_count):
    rrt = ry.PathFinder()
    rrt.setProblem(k, start_state, end_state)
    
    ret = rrt.solve()
    
        
    for i in range(trial_count):
        if ret.feasible:
            return ret
        ret = rrt.solve() 
        
    
    print(ret.feasible) 
    
    return ret


def rrt_viewer(C, ret, attach_state = None, attach = False):
    
    sleep(1)
    if not ret.feasible:
        return
    
    path = ret.x
    
    initial_state = C.getFrameState()
    
    if attach:
        
        C.setJointState(attach_state)
        C.attach(ego1, obj1)
        C.attach(ego2, obj2)
    
    
    for i in path:
        C.setJointState(i)
        C.view()
        sleep(0.3)
    
    C.setFrameState(initial_state)
    
    if attach:
        C.getFrame(obj1).unLink()
        C.getFrame(obj2).unLink()
        
def path_viewer(C, path, attach_index1, attach_index2):
    
    initial_state = C.getFrameState()
    
    C.view()

    sleep(2)
    
    for i in range(0, len(path)):
        
        
        C.setJointState(path[i])
        C.view()
        sleep(0.05)
        
        if i == attach_index1:
            C.attach(ego1, obj1)
        if i == attach_index2:
            C.attach(ego2, obj2)
            
    C.setFrameState(initial_state)
    
    C.getFrame(obj1).unLink()
    C.getFrame(obj2).unLink()
        
        
        

    
    





def komo_whole_planner(c, index):
    temp = ry.Config()
    temp.addConfigurationCopy(c)
    
    temp.delFrame(f"ego{3 - index}")
    temp.delFrame(f"obj{3 - index}")
    
    ego1 = f"ego{index}"
    obj1 = f"obj{index}"
    goal1 = f"goal{index}"
    
    
    first_robot_hold, ret1 = komo_onestate_hold(temp, ego1, obj1)  
    j1_hold = first_robot_hold.getPath()[0]  # birinci tobotun cargonun yanına geldiği state
    
    
    first_robot_move, ret1 = komo_onestate_move(temp, ego1, obj1, goal1)
    j1_moved = first_robot_move.getPath()[0] # birinci robotun goal yanına geldiği state
    
    
    return (j1_hold, j1_moved)






def path_mapper(path, final_size):
    original_path, hold_index = path
    original_path = np.array(original_path)
    path_len = len(original_path)

    # Sample evenly spaced indices, reserving one for final state and one for hold
    steps_to_sample = final_size - 2  # 1 for hold, 1 for final
    sample_indices = np.linspace(0, path_len - 2, steps_to_sample).astype(int)
    sampled_path = original_path[sample_indices]

    # Prepare hold and final states
    
    final_state = original_path[-1].reshape(1, -1)
    
    insert_pos = -1
    
    if hold_index > 0:
        hold_state = original_path[hold_index].reshape(1, -1)
        insert_pos = np.searchsorted(sample_indices, hold_index)
        with_hold = np.insert(sampled_path, insert_pos, hold_state, axis=0)
        final_result = np.vstack([with_hold, final_state])

    else:
        final_result = np.vstack([sampled_path, final_state])
        
    #print("final result: ", final_result)
            
    return final_result, insert_pos


def initial_path_finding(C, serial_no, trial_count):
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    temp.delFrame(f"ego{3 - serial_no}")
    temp.delFrame(f"obj{3 - serial_no}")
    temp.delFrame(f"goal{3 - serial_no}")
    
    ego1 = f"ego{serial_no}"
    obj1 = f"obj{serial_no}" 
    goal1 = f"goal{serial_no}"
    
    
    initial_state = temp.getJointState()
        
    j1_hold, j1_moved = komo_whole_planner(temp, serial_no)
    
    
    ego1_hold_path = rrt_solver(temp, initial_state, j1_hold, trial_count)
    ego1_move_path = rrt_solver(temp, j1_hold, j1_moved, trial_count)
    ego1_final = (np.array([i for i in np.concatenate((ego1_hold_path.x, ego1_move_path.x))]), len(ego1_hold_path.x))
        
    return ego1_final


"""ego1 = "ego1"
ego2 = "ego2"
obj1 = "obj1"
obj2 = "obj2"
goal1 = "goal1"
goal2 = "goal2"

robots = [ego1, ego2]
m_objects = [obj1, obj2]
goals = [goal1, goal2]

goals_reverse = [goal2, goal1]"""


ego1 = "ego1"
ego2 = "ego2"
obj1 = "obj1"
obj2 = "obj2"
goal1 = "goal1"
goal2 = "goal2"

robots = [ego1, ego2]
m_objects = [obj1, obj2]
goals = [goal1, goal2]


C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g", "map4_one_cave.g", "map5_pioneer.g", "map3_corner.g"]
segman_maps = ["pcg-1.g", "pcg-2.g"]

#C.addFile(f"maps/{maps[3]}")


C.addFile(f"segman_maps/real_maps/{segman_maps[1]}")

C.view()

print(C.getJointNames())  # import robotic as ry
import numpy as np

from time import sleep

import math
import sys



def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)

def komo_onestate_hold(C, robot, m_object):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addControlObjective([], 0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)


def komo_onestate_move(C, robot, m_object, goal):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder= 1, enableCollisions=True)
    komo.addModeSwitch([0,1], ry.SY.stable, [robot, m_object], True) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    komo.addObjective([1, 2] , ry.FS.positionDiff, [m_object, goal], ry.OT.eq) 
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)

    

def rrt_solver(k, start_state, end_state, trial_count):
    rrt = ry.PathFinder()
    rrt.setProblem(k, start_state, end_state)
    
    ret = rrt.solve()
    
        
    for i in range(trial_count):
        if ret.feasible:
            return ret
        ret = rrt.solve() 
        
    
    print(ret.feasible) 
    
    return ret


def rrt_viewer(C, ret, attach_state = None, attach = False):
    
    sleep(1)
    if not ret.feasible:
        return
    
    path = ret.x
    
    initial_state = C.getFrameState()
    
    if attach:
        
        C.setJointState(attach_state)
        C.attach(ego1, obj1)
        C.attach(ego2, obj2)
    
    
    for i in path:
        C.setJointState(i)
        C.view()
        sleep(0.3)
    
    C.setFrameState(initial_state)
    
    if attach:
        C.getFrame(obj1).unLink()
        C.getFrame(obj2).unLink()
        
def path_viewer(C, path, attach_index1, attach_index2):
    
    initial_state = C.getFrameState()
    
    C.view()

    sleep(2)
    
    for i in range(0, len(path)):
        
        
        C.setJointState(path[i])
        C.view()
        sleep(0.05)
        
        if i == attach_index1:
            C.attach(ego1, obj1)
        if i == attach_index2:
            C.attach(ego2, obj2)
            
    C.setFrameState(initial_state)
    
    C.getFrame(obj1).unLink()
    C.getFrame(obj2).unLink()
        
        
        

    
    





def komo_whole_planner(c, index):
    temp = ry.Config()
    temp.addConfigurationCopy(c)
    
    temp.delFrame(f"ego{3 - index}")
    temp.delFrame(f"obj{3 - index}")
    
    ego1 = f"ego{index}"
    obj1 = f"obj{index}"
    goal1 = f"goal{index}"
    
    
    first_robot_hold, ret1 = komo_onestate_hold(temp, ego1, obj1)  
    j1_hold = first_robot_hold.getPath()[0]  # birinci tobotun cargonun yanına geldiği state
    
    
    first_robot_move, ret1 = komo_onestate_move(temp, ego1, obj1, goal1)
    j1_moved = first_robot_move.getPath()[0] # birinci robotun goal yanına geldiği state
    
    
    return (j1_hold, j1_moved)






def path_mapper(path, final_size):
    original_path, hold_index = path
    original_path = np.array(original_path)
    path_len = len(original_path)

    # Sample evenly spaced indices, reserving one for final state and one for hold
    steps_to_sample = final_size - 2  # 1 for hold, 1 for final
    sample_indices = np.linspace(0, path_len - 2, steps_to_sample).astype(int)
    sampled_path = original_path[sample_indices]

    # Prepare hold and final states
    
    final_state = original_path[-1].reshape(1, -1)
    
    insert_pos = -1
    
    if hold_index > 0:
        hold_state = original_path[hold_index].reshape(1, -1)
        insert_pos = np.searchsorted(sample_indices, hold_index)
        with_hold = np.insert(sampled_path, insert_pos, hold_state, axis=0)
        final_result = np.vstack([with_hold, final_state])

    else:
        final_result = np.vstack([sampled_path, final_state])
        
    #print("final result: ", final_result)
            
    return final_result, insert_pos


def initial_path_finding(C, serial_no, trial_count):
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    temp.delFrame(f"ego{3 - serial_no}")
    temp.delFrame(f"obj{3 - serial_no}")
    temp.delFrame(f"goal{3 - serial_no}")
    
    ego1 = f"ego{serial_no}"
    obj1 = f"obj{serial_no}" 
    goal1 = f"goal{serial_no}"
    
    
    initial_state = temp.getJointState()
        
    j1_hold, j1_moved = komo_whole_planner(temp, serial_no)
    
    
    ego1_hold_path = rrt_solver(temp, initial_state, j1_hold, trial_count)
    ego1_move_path = rrt_solver(temp, j1_hold, j1_moved, trial_count)
    ego1_final = (np.array([i for i in np.concatenate((ego1_hold_path.x, ego1_move_path.x))]), len(ego1_hold_path.x))
        
    return ego1_final


"""ego1 = "ego1"
ego2 = "ego2"
obj1 = "obj1"
obj2 = "obj2"
goal1 = "goal1"
goal2 = "goal2"

robots = [ego1, ego2]
m_objects = [obj1, obj2]
goals = [goal1, goal2]

goals_reverse = [goal2, goal1]"""


ego1 = "ego1"
ego2 = "ego2"
obj1 = "obj1"
obj2 = "obj2"
goal1 = "goal1"
goal2 = "goal2"

robots = [ego1, ego2]
m_objects = [obj1, obj2]
goals = [goal1, goal2]


C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g", "map4_one_cave.g", "map5_pioneer.g", "map3_corner.g"]
segman_maps = ["pcg-1.g", "pcg-2.g"]

#C.addFile(f"maps/{maps[3]}")


C.addFile(f"segman_maps/real_maps/{segman_maps[0]}")

C.view()

 # ['ego1:0', 'ego1:1', 'ego2:0', 'ego2:1']




['ego1:0', 'ego1:1', 'ego2:0', 'ego2:1']


0

In [10]:
ego1_final = path_mapper(initial_path_finding(C, 1, 5), 60)
ego2_final = path_mapper(initial_path_finding(C, 2, 5), 60)

-- WARNING:kin.cpp:getFrame:415(-1) cannot find frame named 'ego2'
-- WARNING:kin.cpp:getFrame:415(-1) cannot find frame named 'obj2'
-- WARNING:kin.cpp:getFrame:415(-1) cannot find frame named 'ego1'
-- WARNING:kin.cpp:getFrame:415(-1) cannot find frame named 'obj1'


In [11]:
def find_available_point_solve(ego1_final, ego2_final, collision_indices, is_free, search_radius=1.0, num_samples=100):
    ego1_path = ego1_final[0]
    ego2_path = ego2_final[0]

    # Step 1: Find where to start
    first_collision_idx = collision_indices[0]
    start_idx = max(0, first_collision_idx - 5)
    start_point = ego1_path[start_idx]  # [x, y]

    # Step 2: Define helper to check distance to ego2_path
    def is_far_from_ego2(xy, ego2_path, min_dist=0.5):
        return np.all(np.linalg.norm(ego2_path - xy, axis=1) > min_dist)

    # Step 3: Search around the start point
    for _ in range(num_samples):
        # Randomly sample around start_point within search_radius
        angle = np.random.uniform(0, 2*np.pi)
        radius = np.random.uniform(0, search_radius)
        dx = radius * np.cos(angle)
        dy = radius * np.sin(angle)
        candidate = np.array([start_point[0] + dx, start_point[1] + dy])

        if is_free(candidate[0], candidate[1]) and is_far_from_ego2(candidate, ego2_path):
            
            temp = ry.Config()
            temp.addConfigurationCopy(C)
            
            temp.setJointState(np.concatenate((ego1_path[start_idx], ego2_path[start_idx])))            
            
            temp.selectJoints(["ego1"])
            
            temp_new_point_solver = rrt_solver(temp, temp.getJointState(), candidate, 3) 
                        
            if temp_new_point_solver.feasible:
                return temp_new_point_solver, candidate  # found a good point

    # Step 4: If not found, fallback
    print("Warning: No free point found. Returning original point.")
    return start_point


def is_free(x, y):
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    temp.addFrame("frmae").setShape(ry.ST.ssBox, [0.1, 0.1, 0.1, 0]).setPosition([x, y, 0.2]).setContact(1)
    
    temp.computeCollisions()
    r, y = temp.eval(ry.FS.accumulatedCollisions, [])

    del temp
    
    if r > 0:
        return False
    
    return True







In [12]:
# heuristic one -- checking for any collision


def collision_states(ego1_path, ego2_path): 
    
    state_indices = []
    
    for i in range(0, len(ego1_path)):
        
        ego1_loc = ego1_path[i]
        ego2_loc = ego2_path[i]
    
        distance = math.sqrt(pow(abs(ego1_loc[0] - ego2_loc[0]), 2) + pow(abs(ego1_loc[1] - ego2_loc[1]), 2))
                    
        if distance < 0.5:
            state_indices.append(i)
            
    return state_indices

    
def find_another_path_in_collision(collision_states, ego1_final, ego2_final, trial_count):
    
    ego1_path = ego1_final[0]
    ego2_path = ego2_final[0]
        
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    sstt = np.concatenate((ego1_path[ego1_final[1]], ego2_path[ego2_final[1]]))
    temp.setJointState(sstt)
    
    temp.attach(ego1, obj1)
    temp.attach(ego2, obj2)
        
    rrt_again_initial = np.concatenate((ego1_path[collision_states[0] - 3], ego2_path[collision_states[0] - 3]))
    rrt_again_last = np.concatenate((ego1_path[collision_states[-1] + 3], ego2_path[collision_states[-1] + 3]))
    
    
    
    rrt_again = rrt_solver(temp, rrt_again_initial, rrt_again_last, trial_count)
    
    
    del temp

    if rrt_again.feasible:
        current_path = path_mapper((rrt_again.x, -1), len(collision_indices) + 7)
        return current_path

    return False


def manipulate_main(ego1_final, ego2_final, collision_indices, current_path):
    ego1_path, ego1_hold_idx = ego1_final
    ego2_path, ego2_hold_idx = ego2_final

    # Compute range to overwrite
    start = max(collision_indices[0] - 3, 0)
    end = min(collision_indices[-1] + 3, min(len(ego1_path), len(ego2_path), len(current_path[0]) + start + 1))  # assumes ego1, ego2 same length
    
    print("start: ", start)
    print("end:", end)
    print("len:", len(current_path[0]))

    # Replace values in ego1 and ego2 from current_path
    ego1_path[start:end] = current_path[0][0:end - start, :2]   # First 3 values
    ego2_path[start:end] = current_path[0][0:end - start, 2:]   # Last 3 values

    return (ego1_path, ego1_hold_idx), (ego2_path, ego2_hold_idx)


def make_wait(ego1_final, ego2_final, collision_indices):
    ego1_path, ego1_hold_idx = ego1_final
    ego2_path, ego2_hold_idx = ego2_final

    # How many steps to repeat
    wait_len = int(len(collision_indices) / 2) + 6
    insert_idx = collision_indices[0] - 3

    # Get the state to repeat in ego1
    wait_state_ego1 = ego1_path[insert_idx]
    repeated_ego1 = np.repeat(wait_state_ego1.reshape(1, -1), wait_len, axis=0)

    # Insert wait_state into ego1_path at the collision start index
    ego1_path = np.insert(ego1_path, insert_idx, repeated_ego1, axis=0)

    # Extend ego2_path by repeating the last element wait_len times
    last_ego2 = ego2_path[-1]
    repeated_ego2 = np.repeat(last_ego2.reshape(1, -1), wait_len, axis=0)
    ego2_path = np.concatenate((ego2_path, repeated_ego2), axis=0)

    return (ego1_path, ego1_hold_idx), (ego2_path, ego2_hold_idx)


for i in range(1):

    collision_indices = collision_states(ego1_final[0], ego2_final[0])

    print(collision_indices)

    if len(collision_indices) == 0:
        print("problem solved via regular singular rrt ")
        final_path = np.hstack((ego1_final[0], ego2_final[0]))    
        path_viewer(C, final_path, ego1_final[1], ego2_final[1])
        break
        
        #sys.exit()
    
  
    
    


    newPath = find_another_path_in_collision(collision_indices, ego1_final, ego2_final, 20)
    
    if not newPath:
        
        print("wait wait wait ")
        ego1_final_wait, ego2_final_wait = make_wait(ego1_final, ego2_final, collision_indices)
                
        newCollisiion = collision_states(ego1_final_wait[0], ego2_final_wait[0])
        
        if len(newCollisiion) == 0:
            print("problem solved via waiting")
            final_path = np.hstack((ego1_final_wait[0], ego2_final_wait[0]))
            
            path_viewer(C, final_path, ego1_final[1], ego2_final[1])
            continue
            #sys.exit()
            
        else:
            candidate_point = find_available_point_solve(ego1_final, ego2_final, collision_indices, is_free)
            
                
            if isinstance(candidate_point, tuple):
                point_path, free_point = candidate_point

                point_path = np.array(point_path.x)  # make sure it’s a clean numpy array

                point_path = path_mapper((point_path, -1), len(collision_indices) + 10)

                start_idx = max(0, collision_indices[0] - 5)

                # Modify ego1_final
                ego1_path_before = ego1_final[0][:start_idx]
                ego1_path_after = ego1_final[0][collision_indices[-1]+1:]

                new_ego1_path = np.vstack((ego1_path_before, point_path[0], ego1_path_after))
                ego1_final = (new_ego1_path, ego1_final[1])  # preserve timestamps

                print("Updated ego1_final path with detour inserted.")

                # --- Now for ego2_path ---

                # Step 1: Get the element to repeat
                repeat_idx = max(0, collision_indices[0] - 3)
                repeat_element = ego2_final[0][repeat_idx]  # this is a [x, y] vector (or [x,y,z,...])

                # Step 2: Create 5 copies of it
                repeated_elements = np.repeat(repeat_element[np.newaxis, :], 4, axis=0)

                # Step 3: Modify ego2_final path
                ego2_path_before = ego2_final[0][:repeat_idx]
                ego2_path_after = ego2_final[0][repeat_idx:]  # continue normally after repeat_idx

                new_ego2_path = np.vstack((ego2_path_before, repeated_elements, ego2_path_after))

                ego2_final = (new_ego2_path, ego2_final[1])  # again preserve timestamps

                print("Updated ego2_final path by repeating stuck state 5 times.")

                # Combine and view
                final_path = np.hstack((ego1_final[0], ego2_final[0]))
                path_viewer(C, final_path, ego1_final[1], ego1_final[1])

                
            else:
                print("no solution found")
            continue
        

    else:
        
        print("problem solved via finding solution via new multiple rrt")
        
        ego1_final, ego2_final = manipulate_main(ego1_final, ego2_final, collision_indices, newPath)
        final_path = np.hstack((ego1_final[0], ego2_final[0]))
        path_viewer(C, final_path, ego1_final[1], ego2_final[1])
        #sys.exit("problem solved  by selecting differetn collision path")
        continue
    
            
        
        
        
    
    


[]
problem solved via regular singular rrt 


In [15]:
path_viewer(C, final_path, ego1_final[1], ego2_final[1])